### Load transactions
This script loads all transactions from the `transactions` folder. This is about 180days, 2M+ transaction records. FYI: they do not contain PII data, no worries.

It uses `pandas`, the most common data handling library in python.

In [ ]:
import pandas as pd
import glob
import os
from pathlib import Path

# Get all CSV files from the transactions folder
csv_files = glob.glob("transactions/*.csv")

# Read and combine all CSV files
dataframes = []
for file in csv_files:
    try:
        df = pd.read_csv(file)
        # Add a column to track which file each row came from
        df['source_file'] = os.path.basename(file)
        dataframes.append(df)
    except Exception as e:
        print(f"Error loading {file}: {e}")

# Combine all dataframes
if dataframes:
    transactions = pd.concat(dataframes, ignore_index=True)
    print(f"Loaded {len(transactions):,} transactions")
    

In [ ]:
# Convert Creation Date to datetime for proper date operations
transactions['Creation Date'] = pd.to_datetime(transactions['Creation Date'])

display(transactions.head())

### User Names per Account with number of Transactions

In [ ]:
# Analysis: Transactions per Account and User Name
transactions_by_account_user = transactions.groupby(['Account', 'User Name']).size().reset_index(name='Transaction Count')

# Sort by Account first, then by Transaction Count descending
transactions_by_account_user = transactions_by_account_user.sort_values(['Account', 'Transaction Count'], ascending=[True, False])

print(f"Total unique Account-User combinations: {len(transactions_by_account_user)}")
print("\nTransactions per Account and User Name:")
display(transactions_by_account_user)


In [ ]:
# Write account analysis to CSV file
transactions_by_account_user.to_csv('out/user-transaction-count.csv', index=False)

### Merchant Accounts with User Name and Transaction Dates
This analysis shows all accounts with their user names, transaction counts, and the date range of their transactions.

In [ ]:
# Analysis: All Accounts with User Names, transaction count, and date range
account_analysis = transactions.groupby(['Account', 'User Name']).agg({
    'PSP Reference': 'count',  # Count of transactions
    'Creation Date': ['min', 'max']  # Earliest and latest dates
}).reset_index()

# Flatten the column names
account_analysis.columns = ['Account', 'User Name', 'Transaction Count', 'Earliest Transaction', 'Latest Transaction']

# Sort by Account, then by Transaction Count descending
account_analysis = account_analysis.sort_values(['Account', 'Transaction Count'], ascending=[True, False])

# Format the dates for better readability
account_analysis['Earliest Transaction'] = account_analysis['Earliest Transaction'].dt.strftime('%Y-%m-%d %H:%M:%S')
account_analysis['Latest Transaction'] = account_analysis['Latest Transaction'].dt.strftime('%Y-%m-%d %H:%M:%S')

print(f"Total unique Account-User combinations: {len(account_analysis)}")
print(f"Date range of all transactions: {transactions['Creation Date'].min().strftime('%Y-%m-%d')} to {transactions['Creation Date'].max().strftime('%Y-%m-%d')}")
print("\nComplete Account Analysis:")
display(account_analysis)


In [ ]:
# Write account analysis to CSV file
account_analysis.to_csv('out/accounts-with-users.csv', index=False)

### Usernames and their last use
Just a list of all API users and when there last transaction was.

In [ ]:
# Analysis: All Accounts with User Names, transaction count, and date range
user_analysis = transactions.groupby(['User Name']).agg({
    'PSP Reference': 'count',  # Count of transactions
    'Creation Date': ['min', 'max']  # Earliest and latest dates
}).reset_index()

# Flatten the column names
user_analysis.columns = ['User Name', 'Transaction Count', 'Earliest Transaction', 'Latest Transaction']

# Sort by Account, then by Transaction Count descending
user_analysis = user_analysis.sort_values(['Transaction Count', 'Latest Transaction'], ascending=[False, False])

# Format the dates for better readability
user_analysis['Earliest Transaction'] = user_analysis['Earliest Transaction'].dt.strftime('%Y-%m-%d %H:%M:%S')
user_analysis['Latest Transaction'] = user_analysis['Latest Transaction'].dt.strftime('%Y-%m-%d %H:%M:%S')

print(f"Total active users: {len(user_analysis)}")
print(f"Date range of all transactions: {transactions['Creation Date'].min().strftime('%Y-%m-%d')} to {transactions['Creation Date'].max().strftime('%Y-%m-%d')}")
display(user_analysis)

In [ ]:
# Write account analysis to CSV file
user_analysis.to_csv('out/active-users.csv', index=False)

### Lets plot some graphs

In [ ]:
CHF = 1.0671047
DKK = 0.13404138
EUR = 1
GBP = 1.1874339
MYR = 0.20705396
USD = 0.87635219

### Absolute income per currency

In [ ]:
# Analysis: Income per month by currency
import matplotlib.pyplot as plt

# Filter for income transactions (positive values and successful statuses)
income_transactions = transactions[
    (transactions['Value'] > 0) & 
    (transactions['Status'].isin(['SettledBulk', 'Settled', 'Authorised']))
].copy()

# Extract year-month from Creation Date
income_transactions['Year-Month'] = income_transactions['Creation Date'].dt.to_period('M')

# Group by Currency and Year-Month, sum the values
monthly_income_by_currency = income_transactions.groupby(['Currency', 'Year-Month'])['Value'].sum().reset_index()

# Convert Year-Month back to datetime for plotting
monthly_income_by_currency['Date'] = monthly_income_by_currency['Year-Month'].dt.to_timestamp()

# Create the plot
plt.figure(figsize=(15, 8))

# Get unique currencies
currencies = monthly_income_by_currency['Currency'].unique()

# Create a line plot for each currency
for currency in currencies:
    currency_data = monthly_income_by_currency[monthly_income_by_currency['Currency'] == currency]
    plt.plot(currency_data['Date'], currency_data['Value'], marker='o', linewidth=2, label=currency)

plt.title('Monthly Income by Currency', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Income Amount', fontsize=12)
plt.legend(title='Currency', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

# Format y-axis to show currency amounts nicely
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))

plt.tight_layout()
plt.show()

# Display summary statistics
print(f"Total income transactions: {len(income_transactions):,}")
print(f"Date range: {income_transactions['Creation Date'].min().strftime('%Y-%m-%d')} to {income_transactions['Creation Date'].max().strftime('%Y-%m-%d')}")
print(f"Currencies with income: {', '.join(sorted(currencies))}")
print(f"\nTotal income by currency:")
total_by_currency = income_transactions.groupby('Currency')['Value'].sum().sort_values(ascending=False)
for currency, total in total_by_currency.items():
    print(f"  {currency}: {total:,.2f}")

### Relative income per currency

In [ ]:
# Create relative income graph where EUR average = 1
import matplotlib.pyplot as plt
import numpy as np

# Exchange rates to EUR (from Cell 11)
exchange_rates = {
    'CHF': 1.0671047,
    'DKK': 0.13404138,
    'EUR': 1,
    'GBP': 1.1874339,
    'MYR': 0.20705396,
    'USD': 0.87635219
}

# Convert all incomes to EUR equivalent
monthly_income_eur = monthly_income_by_currency.copy()
monthly_income_eur['Value_EUR'] = monthly_income_eur.apply(
    lambda row: row['Value'] * exchange_rates[row['Currency']], axis=1
)

# Calculate average EUR income across all months
eur_data = monthly_income_eur[monthly_income_eur['Currency'] == 'EUR']
eur_average = eur_data['Value_EUR'].mean()

# Create relative values where EUR average = 1
monthly_income_eur['Relative_Value'] = monthly_income_eur['Value_EUR'] / eur_average

# Create the plot
plt.figure(figsize=(15, 10))

# Get unique currencies and create colors
currencies = sorted(monthly_income_eur['Currency'].unique())
colors = plt.cm.Set3(np.linspace(0, 1, len(currencies)))

# Create a line plot for each currency
for i, currency in enumerate(currencies):
    currency_data = monthly_income_eur[monthly_income_eur['Currency'] == currency]
    plt.plot(currency_data['Date'], currency_data['Relative_Value'], 
             marker='o', linewidth=2.5, markersize=6, 
             label=currency, color=colors[i])

# Add horizontal line at y=1 for reference (EUR average)
plt.axhline(y=1, color='red', linestyle='--', alpha=0.7, linewidth=1, 
            label='EUR Average = 1')

plt.title('Relative Monthly Income by Currency\n(EUR Average = 1.0)', 
          fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Relative Income (EUR Average = 1)', fontsize=12)
plt.legend(title='Currency', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

# Format y-axis 
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.2f}'))

plt.tight_layout()
plt.show()

# Display summary statistics
print("Exchange rates used (to EUR):")
for currency, rate in exchange_rates.items():
    print(f"  {currency}: {rate:.6f}")

print(f"\nEUR average monthly income: {eur_average:,.2f}")
print(f"\nAverage relative income by currency (EUR = 1.0):")
relative_averages = monthly_income_eur.groupby('Currency')['Relative_Value'].mean().sort_values(ascending=False)
for currency, avg_relative in relative_averages.items():
    print(f"  {currency}: {avg_relative:.3f}")

### The 2 graphs next to each other

# Display both absolute and relative income graphs side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Left plot: Absolute income by currency (from Cell 10)
currencies = monthly_income_by_currency['Currency'].unique()
colors = plt.cm.Set3(np.linspace(0, 1, len(currencies)))

for i, currency in enumerate(sorted(currencies)):
    currency_data = monthly_income_by_currency[monthly_income_by_currency['Currency'] == currency]
    ax1.plot(currency_data['Date'], currency_data['Value'], 
             marker='o', linewidth=2.5, markersize=6, 
             label=currency, color=colors[i])

ax1.set_title('Monthly Income by Currency\n(Absolute Values)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Month', fontsize=12)
ax1.set_ylabel('Income Amount', fontsize=12)
ax1.legend(title='Currency', bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))

# Right plot: Relative income by currency (from Cell 12)
for i, currency in enumerate(sorted(currencies)):
    currency_data = monthly_income_eur[monthly_income_eur['Currency'] == currency]
    ax2.plot(currency_data['Date'], currency_data['Relative_Value'], 
             marker='o', linewidth=2.5, markersize=6, 
             label=currency, color=colors[i])

# Add horizontal line at y=1 for reference (EUR average)
ax2.axhline(y=1, color='red', linestyle='--', alpha=0.7, linewidth=1, 
            label='EUR Average = 1')

ax2.set_title('Relative Monthly Income by Currency\n(EUR Average = 1.0)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Month', fontsize=12)
ax2.set_ylabel('Relative Income (EUR Average = 1)', fontsize=12)
ax2.legend(title='Currency', bbox_to_anchor=(1.05, 1), loc='upper left')
ax2.grid(True, alpha=0.3)
ax2.tick_params(axis='x', rotation=45)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.2f}'))

plt.tight_layout()
plt.show()

# Display comparison summary
print("=== COMPARISON SUMMARY ===")
print(f"Total currencies analyzed: {len(currencies)}")
print(f"Date range: {monthly_income_by_currency['Date'].min().strftime('%Y-%m')} to {monthly_income_by_currency['Date'].max().strftime('%Y-%m')}")

print(f"\nAbsolute totals by currency:")
for currency in sorted(currencies):
    currency_total = monthly_income_by_currency[monthly_income_by_currency['Currency'] == currency]['Value'].sum()
    print(f"  {currency}: {currency_total:,.2f}")

print(f"\nRelative averages by currency (EUR = 1.0):")
for currency, avg_relative in relative_averages.items():
    print(f"  {currency}: {avg_relative:.3f}")